In [1]:
from pygraylog.graylogapi import GraylogAPI
import json
import sys
import numpy as np
import pandas as pd

In [2]:
log_level = 0

In [3]:
graylog_con={
        "user":"merger",
        "password":"",
        "url":"http://graylog-ui.roamability.com:9000/api"
    }

In [4]:
graylog_con={
        "user":"pavel",
        "password":"O777ver7k1",
        "url":"http://graylog-ui.roamability.com:9000/api"
    }

In [5]:
api = GraylogAPI(
    graylog_con["url"],
    graylog_con["user"], 
    graylog_con["password"])

In [6]:
import datetime
#format like 2017-03-06T16:28
dateFrom = datetime.datetime.strptime('2018-01-12T05:00', '%Y-%m-%dT%H:%M')
dateTo = datetime.datetime.strptime('2018-01-12T12:00', '%Y-%m-%dT%H:%M')
#query='protocol:ss7 AND opCode_localValue:2 AND TCAP_type:Begin AND e212_imsi:425191201068102 AND tcap_otid:"a2:2a:0c:40"'
query='protocol:ss7 AND (tcap_otid:"a2:2a:0c:40" OR tcap_dtid:"a2:2a:0c:40")'

In [7]:
gres=api.search.universal.absolute.get(query=query, from_=dateFrom, to=dateTo)

In [8]:
def outLog(message, level=1):
    if(level <= log_level):
        print (message)
        sys.stdout.flush()

In [9]:
def LoadGraylogResult(exportsearch_results):
        jsonResult = json.loads(exportsearch_results)
        outputResult = list()
        i=0
        if "total_results" in jsonResult:
            outLog("Graylog search found {} messages".format(jsonResult["total_results"]),0)

        # Get the results and store them
            for result in jsonResult["messages"]:  
                if "message" in  result and isinstance(result["message"], dict):
                    outputResult.append(result["message"])
                    i+=1
                    if i%100==0: outLog("loaded {} records from Graylog".format(i),0)
            outLog("Graylog search load {} messages".format(len(outputResult)),0)
        else:
            outLog("Graylog search error: {}".format(exportsearch_results),0)

        return outputResult

In [10]:
pgres=LoadGraylogResult(gres)

Graylog search found 59 messages
Graylog search load 59 messages


In [11]:
data=pgres

In [12]:
columns=list(data[0].keys())
values = list(data[0].values())
arr_len = len(values)
pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)

,protocol,gl2_source_collector,sccp_calling_ssn,tcap_dtid,opCode_localValue,sccp_message_type,m3ua_DPC,file,message,sccp_called_nai,...,offset,sccp_called_ri,gl2_source_node,m3ua_OPC,sccp_called_digits,TCAP_type,streams,sccp_calling_ri,_id,frame_number
0,ss7,b3d2019c-476e-439e-8764-043b24bcda9d,6,a2:2a:0c:40,56,0x00000009,800,/mnt/tracing1ch/sigtran-public/sigtran-public_...,12/01/2018 11:30:27.387814 \tprotocol=ss7\tfil...,4,...,6172175,0,be115c61-9b16-42d7-810c-8f0e4af6a69c,781,919434399505,End,[000000000000000000000001],0,96eac9ef-f785-11e7-8ae9-00505698f35d,20803


In [13]:
columns=list(data[0].keys())
values = list(data[0].values())
arr_len = len(values)
df=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
for i in range(len(data)-1):
    columns=list(data[i+1].keys())
    values = list(data[i+1].values())
    arr_len = len(values)
    df1=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
    df=pd.concat([df, df1],axis=0,join='outer')
df=df.reset_index(drop=True)
df['tcap_otid']=df['tcap_otid'].fillna(0)
df['tcap_dtid']=df['tcap_dtid'].fillna(0)
df.head(3)

,TCAP_type,_id,collector_node_id,e164_msisdn,e212_imsi,e212_mcc,e212_mnc,facility,file,frame_number,...,sccp_class,sccp_message_type,source,streams,tags,tcap_application_context_name,tcap_dtid,tcap_otid,timestamp,type
0,End,96eac9ef-f785-11e7-8ae9-00505698f35d,graylog-collector-sidecar,NaN,NaN,NaN,NaN,filebeat,/mnt/tracing1ch/sigtran-public/sigtran-public_...,20803,...,0,0x00000009,admin,[000000000000000000000001],[tracer],0.4.0.0.1.0.14.2,a2:2a:0c:40,0,2018-01-12T10:30:27.387Z,pcap
1,Begin,b890ae26-f785-11e7-8ae9-00505698f35d,graylog-collector-sidecar,NaN,425019612539521,425,1,filebeat,/mnt/tracing2ch/sigtran-public/sigtran-public_...,9943,...,1,0x00000009,admin,[000000000000000000000001],[tracer],0.4.0.0.1.0.14.2,0,a2:2a:0c:40,2018-01-12T10:30:27.375Z,pcap
2,Begin,b85a82f2-f785-11e7-8ae9-00505698f35d,graylog-collector-sidecar,46731726312,240400000038251,240,40,filebeat,/mnt/tracing2ch/sigtran-public/sigtran-public_...,7046,...,NaN,0x0000000a,admin,[000000000000000000000001],[tracer],0.4.0.0.1.0.25.3,0,a2:2a:0c:40,2018-01-12T10:29:55.578Z,pcap


In [14]:
df[[
    'm3ua_OPC','m3ua_DPC',
    'sccp_calling_nai','sccp_calling_digits','sccp_calling_ri','sccp_calling_ssn',
    'sccp_called_nai','sccp_called_digits','sccp_called_ri','sccp_called_ssn',
    'sccp_class','sccp_message_type','protocol_name','protocol','protocol_version','opCode_localValue','TCAP_type',
    'tcap_otid','tcap_dtid',
    'e164_msisdn','e212_imsi']].head()

,m3ua_OPC,m3ua_DPC,sccp_calling_nai,sccp_calling_digits,sccp_calling_ri,sccp_calling_ssn,sccp_called_nai,sccp_called_digits,sccp_called_ri,sccp_called_ssn,...,sccp_message_type,protocol_name,protocol,protocol_version,opCode_localValue,TCAP_type,tcap_otid,tcap_dtid,e164_msisdn,e212_imsi
0,781,800,4,97254120634,0,6,4,919434399505,0,7,...,0x00000009,map,ss7,2,56,End,0,a2:2a:0c:40,NaN,NaN
1,801,782,4,919434399505,0,7,4,972549612539521,0,6,...,0x00000009,map,ss7,NaN,56,Begin,a2:2a:0c:40,0,NaN,425019612539521
2,1962,2506,4,46731726312,0,8,4,46731726312,0,8,...,0x0000000a,SCCP,ss7,NaN,44,Begin,a2:2a:0c:40,0,46731726312,240400000038251
3,2506,1962,4,46731726312,0,8,4,46731726312,0,8,...,0x00000009,GSM,ss7,NaN,44,Begin,a2:2a:0c:40,0,46731726312,240400000038251
4,7138,701,4,972559900050,0,6,4,972559900040,0,7,...,0x00000009,map,ss7,NaN,56,End,0,a2:2a:0c:40,NaN,NaN


In [15]:
#Функция для сборки транзакции

def xdr_compile(otid,ind,df_xdr,df):
    dtid=111
    for i in range(ind-1,-1,-1):
        if df.iloc[i]['tcap_dtid']==otid and df.iloc[i]['tcap_otid']==0 and df.iloc[i]['TCAP_type']=='End':
            values = list(df.iloc[i].values)
            arr_len = len(values)
            df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
            df_xdr=pd.concat([df_xdr, df_temp],axis=0,join='outer')
            return df_xdr
        elif df.iloc[i]['tcap_dtid']==otid and df.iloc[i]['TCAP_type']<>'Begin' and df.iloc[i]['tcap_otid']<>dtid:
            dtid=df.iloc[i]['tcap_otid']
            values = list(df.iloc[i].values)
            arr_len = len(values)
            df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
            df_xdr=pd.concat([df_xdr, df_temp],axis=0,join='outer')
        elif df.iloc[i]['TCAP_type']<>'Begin' and (df.iloc[i]['tcap_dtid']==otid or df.iloc[i]['tcap_otid']==otid or df.iloc[i]['tcap_dtid']==dtid or df.iloc[i]['tcap_otid']==dtid):
            values = list(df.iloc[i].values)
            arr_len = len(values)
            df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
            df_xdr=pd.concat([df_xdr, df_temp],axis=0,join='outer')            
    return df_xdr

In [16]:
#Сборка датафрейма XDR

columns=list(df.keys())
df_xdr_res=pd.DataFrame(columns=columns)
for i in range(len(df)-1,-1,-1):
    if df.iloc[i]['TCAP_type']=='Begin' and (df.iloc[i]['e212_imsi']=='724540540280711'):
        values = list(df.iloc[i].values)
        arr_len = len(values)
        df_temp=pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)
        #df_xdr=pd.concat([df_xdr, df_temp],axis=0,join='outer')
        #df.drop([i],inplace=True)
        df_xdr_res=pd.concat([df_xdr_res,xdr_compile(df.iloc[i]['tcap_otid'],i+1,df_temp,df)],axis=0,join='outer')
df_xdr_res=df_xdr_res.reset_index(drop=True)
df_xdr_res[['timestamp',
    'm3ua_OPC','m3ua_DPC',
    'sccp_calling_nai','sccp_calling_digits','sccp_calling_ri','sccp_calling_ssn',
    'sccp_called_nai','sccp_called_digits','sccp_called_ri','sccp_called_ssn',
    'sccp_class','sccp_message_type','protocol_name','protocol','protocol_version','opCode_localValue','TCAP_type',
    'tcap_otid','tcap_dtid',
    'e164_msisdn','e212_imsi']]

,timestamp,m3ua_OPC,m3ua_DPC,sccp_calling_nai,sccp_calling_digits,sccp_calling_ri,sccp_calling_ssn,sccp_called_nai,sccp_called_digits,sccp_called_ri,...,sccp_message_type,protocol_name,protocol,protocol_version,opCode_localValue,TCAP_type,tcap_otid,tcap_dtid,e164_msisdn,e212_imsi
0,2018-01-12T10:30:27.375Z,801,782,4,919434399505,0,7,4,972549612539521,0,...,0x00000009,map,ss7,NaN,56,Begin,a2:2a:0c:40,0,NaN,425019612539521
1,2018-01-12T10:30:27.387Z,781,800,4,97254120634,0,6,4,919434399505,0,...,0x00000009,map,ss7,2,56,End,0,a2:2a:0c:40,NaN,NaN
